<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/BERT_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys

!git clone https://github.com/google-research/bert bert_repo
if not 'bert_repo' in sys.path:
  sys.path += ['bert_repo']


In [ ]:
!pip install -r /content/bert_repo/requirements.txt -q
!pip install -U protobuf -q

In [4]:
import tensorflow as tf; print(tf.__version__)

2.20.0


In [1]:
!pip install transformers -q -U
!pip install datasets -q

ERROR: git+https://github.com/google-research/bert.git does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 107.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.9/73.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.3/235.3 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.5/213.5

In [3]:
# Combined JAX BERT Fine-tuning and Prediction Code

# --- Imports ---
import jax
import jax.numpy as jnp
import optax
from flax.training import train_state
import flax.linen as nn
from functools import partial
import os
from transformers import FlaxBertForSequenceClassification, AutoTokenizer, AutoConfig # Import necessary transformers components
from datasets import load_dataset # Import datasets library
import warnings
warnings.filterwarnings("ignore")

# Ensure JAX is using the TPU
print("JAX devices:", jax.devices())

JAX devices: [TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0), TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1), TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0), TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1), TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0), TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1), TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0), TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)]


In [ ]:
# Combined JAX BERT Fine-tuning and Prediction Code

# --- Parameters (Ensure these are set from previous cells or define here) ---
# These variables should ideally be defined in previous cells and available in the environment.
# If not, you might need to manually set them here based on your configuration.
# Example placeholders (replace with actual values if necessary):
BERT_MODEL_HUB = "google/bert-base-uncased" # Using huggingface model name for transformers
TASK = "mrpc" # Lowercase task name for datasets library # Corrected task name to lowercase
MAX_SEQ_LENGTH = 128
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 8
PREDICT_BATCH_SIZE = 8
SAVE_SUMMARY_STEPS = 500 # For logging frequency

# Assuming these variables are defined from earlier in the notebook:
# Check if variables exist and print their values if they do, otherwise use defaults or print a message
# This helps in debugging if previous cells weren't run
task_name_to_print = TASK if 'TASK' in locals() else "N/A (TASK not defined)"
max_seq_length_to_print = MAX_SEQ_LENGTH if 'MAX_SEQ_LENGTH' in locals() else "N/A (MAX_SEQ_LENGTH not defined)"
learning_rate_to_print = LEARNING_RATE if 'LEARNING_RATE' in locals() else "N/A (LEARNING_RATE not defined)"
train_batch_size_to_print = TRAIN_BATCH_SIZE if 'TRAIN_BATCH_SIZE' in locals() else "N/A (TRAIN_BATCH_SIZE not defined)"
eval_batch_size_to_print = EVAL_BATCH_SIZE if 'EVAL_BATCH_SIZE' in locals() else "N/A (EVAL_BATCH_SIZE not defined)"
predict_batch_size_to_print = PREDICT_BATCH_SIZE if 'PREDICT_BATCH_SIZE' in locals() else "N/A (PREDICT_BATCH_SIZE not defined)"
num_train_epochs_to_print = NUM_TRAIN_EPOCHS if 'NUM_TRAIN_EPOCHS' in locals() else "N/A (NUM_TRAIN_EPOCHS not defined)"


print(f"Task: {task_name_to_print}, Max Seq Length: {max_seq_length_to_print}, Learning Rate: {learning_rate_to_print}")
print(f"Train Batch Size: {train_batch_size_to_print}, Eval Batch Size: {eval_batch_size_to_print}, Predict Batch Size: {predict_batch_size_to_print}")
print(f"Num Train Epochs: {num_train_epochs_to_print}")


# --- Data Loading and Preprocessing ---
# Use the datasets library to load the GLUE MRPC dataset
try:
    print("Loading dataset...")
    raw_datasets = load_dataset("glue", TASK) # Use the TASK variable (now lowercase)
    print("Dataset loaded successfully.")
except Exception as e:
    print(f"Failed to load dataset: {e}")
    # Fallback or error handling if dataset loading fails

# Load the tokenizer
# Use AutoTokenizer to automatically load the correct tokenizer for the BERT model
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased') # Using bert-base-uncased as in the original notebook

# Define the preprocessing function
def preprocess_function(examples):
    # Tokenize the sentences
    # Use truncation=True to truncate sequences longer than MAX_SEQ_LENGTH
    # Use padding='max_length' to pad sequences shorter than MAX_SEQ_LENGTH
    # Use return_tensors='np' for NumPy arrays or 'tf' for TensorFlow tensors
    # We will return as 'np' tensors since the dataset will be converted to tf.data.Dataset later
    return tokenizer(
        examples['sentence1'],
        examples['sentence2'],
        truncation=True,
        padding='max_length',
        max_length=MAX_SEQ_LENGTH,
        return_token_type_ids=True, # Include token_type_ids
        return_attention_mask=True, # Include attention_mask
        return_tensors='np' # Return NumPy arrays instead of TensorFlow tensors
    )

# Apply the preprocessing function to the entire dataset
# Check if raw_datasets is defined before mapping
if 'raw_datasets' in locals():
    print("Tokenizing dataset...")
    tokenized_dataset = raw_datasets.map(
        preprocess_function,
        batched=True # Process in batches for efficiency
    )
    print("Dataset tokenized.")

    # Rename the 'label' column to 'labels' to match the model's expected input
    tokenized_dataset = tokenized_dataset.rename_column("label", "labels")

    # Remove original text columns if no longer needed to save memory
    tokenized_dataset = tokenized_dataset.remove_columns(["sentence1", "sentence2", "idx"])

    # Convert the tokenized datasets to TensorFlow datasets
    # This is useful for easy batching and shuffling
    train_dataset = tokenized_dataset["train"].to_tf_dataset(
        columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        shuffle=True, # Shuffle training data
        batch_size=TRAIN_BATCH_SIZE,
        collate_fn=None, # Use default collate function
        drop_remainder=True # Drop remainder for fixed batch size in pmap
    )

    eval_dataset = tokenized_dataset["validation"].to_tf_dataset(
        columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        shuffle=False, # No shuffling for evaluation
        batch_size=EVAL_BATCH_SIZE,
        collate_fn=None,
        drop_remainder=True # Drop remainder for fixed batch size in pmap
    )

    # Determine number of training and warmup steps
    # num_train_examples is needed to calculate num_train_steps
    # Use the size of the tokenized training dataset
    num_train_examples = len(tokenized_dataset["train"])
    num_train_steps = int(num_train_examples / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
    WARMUP_PROPORTION = 0.1 # Assuming this value from original notebook
    num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

    print(f"Number of training examples: {num_train_examples}")
    print(f"Number of training steps: {num_train_steps}")
    print(f"Number of warmup steps: {num_warmup_steps}")

else:
    print("Skipping tokenization and dataset conversion due to dataset loading failure.")


# --- Model Loading ---
# Load a pre-trained bert-base-uncased model optimized for sequence classification using Flax
# This requires a compatible version of transformers, jax, and jaxlib
print("Loading pre-trained BERT model...")
# Ensure num_labels is defined - it should be 2 for MRPC
if 'raw_datasets' in locals() and raw_datasets is not None and 'label' in raw_datasets['train'].features:
     # Infer num_labels from the dataset features if dataset loaded successfully
     label_list = raw_datasets['train'].features['label'].names
     num_labels = len(label_list)
     print(f"Setting num_labels = {num_labels} from dataset features.")
elif 'num_labels' not in locals():
     # Default to 2 for MRPC if dataset loading failed and num_labels not defined
     num_labels = 2
     print(f"Assuming num_labels = {num_labels} for MRPC task (dataset loading failed).")


try:
    # Check if num_labels was successfully determined
    if 'num_labels' in locals():
        model = FlaxBertForSequenceClassification.from_pretrained(
            'bert-base-uncased',
            num_labels=num_labels,
            dtype=jnp.float32 # Specify dtype for JAX/TPU compatibility
        )
        params = model.params # Access parameters from the model object
        print("Pre-trained BERT model and parameters loaded successfully.")
    else:
        print("Cannot load model: num_labels could not be determined.")

except ImportError as e:
    print(f"Failed to load model due to library compatibility issue: {e}")
    print("Please ensure you have compatible versions of jax, jaxlib, and transformers installed.")
    # You might need to install specific versions, e.g., !pip install jax==0.4.23 jaxlib==0.4.23 transformers==4.30.0 -q
except Exception as e:
    print(f"An unexpected error occurred during model loading: {e}")
    # Handle other potential errors during model loading

# --- Optimizer and Train State ---
# Define the learning rate schedule and optimizer
# Using optax.join_schedules for linear warmup and linear decay
print("Setting up optimizer...")

# Check if num_train_steps and num_warmup_steps are defined from Data Loading step
if 'num_train_steps' in locals() and 'num_warmup_steps' in locals():
    # Define linear warmup schedule
    warmup_schedule = optax.linear_schedule(
        init_value=0.,
        end_value=LEARNING_RATE,
        transition_steps=num_warmup_steps
    )

    # Define linear decay schedule (starts from LEARNING_RATE and decays to 0)
    # The decay happens over the remaining steps after warmup
    decay_steps_calc = num_train_steps - num_warmup_steps
    decay_schedule = optax.linear_schedule(
        init_value=LEARNING_RATE,
        end_value=0.,
        transition_steps=decay_steps_calc if decay_steps_calc > 0 else 1 # Ensure transition_steps is at least 1
    )

    # Join the two schedules
    # The schedule switches from warmup to decay after num_warmup_steps
    learning_rate_schedule = optax.join_schedules(
        schedules=[warmup_schedule, decay_schedule],
        boundaries=[num_warmup_steps]
    )


    # AdamW is a common optimizer used with BERT
    optimizer = optax.adamw(
        learning_rate=learning_rate_schedule,
        weight_decay=0.01 # Common weight decay for BERT
    )

    # Create the training state
    # This will hold the model parameters, optimizer state, and a PRNG key
    # The model and params should be loaded successfully in the 'Model Loading' section.
    # If model or params are not defined, the model loading failed.
    if 'model' in locals() and 'params' in locals():
        # We need a PRNG key for parameter initialization (although using pretrained params here)
        # and for dropout.
        key = jax.random.PRNGKey(0)
        # Use a consistent key for dropout across training steps, folding in the step number
        dropout_key = jax.random.fold_in(key, 0) # Initial dropout key

        state = train_state.TrainState.create(
            apply_fn=model.__call__,
            params=params, # Use the loaded pretrained parameters
            tx=optimizer
        )

        # Replicate the training state for pmap
        state = jax.device_put_replicated(state, jax.local_devices())

        print("Optimizer and training state created and replicated.")
    else:
        print("Model or parameters not loaded, cannot create optimizer and train state.")
else:
    print("Skipping optimizer and train state setup: num_train_steps or num_warmup_steps not defined.")


# --- JAX Training Step Function ---
# Define the loss function
def compute_loss(logits, labels):
    # Use sparse_softmax_cross_entropy as labels are integers
    # Ensure one_hot encoding matches the number of classes (num_labels)
    # Check if num_labels is defined before using it
    if 'num_labels' in locals():
        loss = optax.softmax_cross_entropy(logits, jax.nn.one_hot(labels, num_classes=num_labels))
        return jnp.mean(loss)
    else:
        print("Cannot compute loss: num_labels not defined.")
        return jnp.nan # Return NaN or handle error appropriately

# Define the training step function
# This function will compute the loss and gradients and update the model parameters.
@partial(jax.jit) # Removed static_argnums
def train_step(state, batch, dropout_key):
    # Split dropout key for each step
    new_dropout_key = jax.random.fold_in(dropout_key, state.step)

    def loss_fn(params):
        # Model forward pass
        # Check if model and num_labels are defined before applying
        if 'model' in locals() and 'num_labels' in locals():
            logits = state.apply_fn(
                batch['input_ids'],
                batch['attention_mask'],
                batch['token_type_ids'],
                params=params,
                deterministic=False, # Enable dropout during training
                rngs={'dropout': new_dropout_key} # Pass dropout key
            )
            loss = compute_loss(logits, batch['labels'])
            return loss, logits # Return logits along with the loss
        else:
            print("Cannot compute loss in train_step: model or num_labels not defined.")
            return jnp.nan, None

    # Check if state and loss_fn are defined before computing gradients
    if 'state' in locals() and 'loss_fn' in locals():
        # Compute gradients
        grad_fn = jax.grad(lambda params: loss_fn(params)[0]) # Compute gradient only w.r.t. loss
        grads = grad_fn(state.params)

        # Get loss and logits from loss_fn call for accuracy calculation
        loss, logits = loss_fn(state.params)

        # Update parameters
        new_state = state.apply_gradients(grads=grads)

        # Compute training accuracy (optional, for monitoring)
        # Check if num_labels is defined and logits is not None
        if 'num_labels' in locals() and logits is not None:
            predicted_labels = jnp.argmax(logits, axis=-1)
            accuracy = jnp.mean(predicted_labels == batch['labels'])
        else:
            print("Cannot compute accuracy in train_step: num_labels not defined.")
            accuracy = jnp.nan

        return new_state, loss, accuracy, new_dropout_key
    else:
        print("Skipping train_step execution: state or loss_fn not defined.")
        # Return dummy values or handle appropriately
        return state, jnp.nan, jnp.nan, new_dropout_key


# pmap the training step for parallel execution across TPU cores
# Check if 'state' is defined before pmapping
if 'state' in locals():
    pmapped_train_step = jax.pmap(train_step, axis_name='batch')
    print("JAX training step function defined and pmapped.")
else:
    print("Train state not created, cannot pmap training step.")


# --- JAX Evaluation Step Function ---
# Define the evaluation step function
@partial(jax.jit) # Removed static_argnums
def eval_step(state, batch): # Removed dropout_key parameter here
    def loss_fn(params):
         # Model forward pass in evaluation mode (deterministic=True, no dropout)
        # Check if model and num_labels are defined before applying
        if 'model' in locals() and 'num_labels' in locals():
            logits = state.apply_fn(
                batch['input_ids'],
                batch['attention_mask'],
                batch['token_type_ids'],
                params=state.params, # Use state.params for evaluation
                deterministic=True, # Explicitly pass deterministic as keyword argument
            )
            loss = compute_loss(logits, batch['labels'])
            return loss, logits # Return logits for accuracy calculation
        else:
            print("Cannot compute loss in eval_step: model or num_labels not defined.")
            return jnp.nan, None

    # Check if state and loss_fn are defined before executing
    if 'state' in locals() and 'loss_fn' in locals():
        loss, logits = loss_fn(state.params)

        # Compute evaluation metrics
        # Check if num_labels is defined and logits is not None
        if 'num_labels' in locals() and logits is not None:
            predicted_labels = jnp.argmax(logits, axis=-1)
            accuracy = jnp.mean(predicted_labels == batch['labels'])
        else:
             print("Cannot compute metrics in eval_step: num_labels not defined or logits is None.")
             predicted_labels = None
             accuracy = jnp.nan


        return loss, accuracy, predicted_labels # Return predicted labels for potential inspection
    else:
        print("Skipping eval_step execution: state or loss_fn not defined.")
        return jnp.nan, jnp.nan, None


# pmap the evaluation step for parallel execution
# Check if 'state' is defined before pmapping
if 'state' in locals():
    pmapped_eval_step = jax.pmap(eval_step, axis_name='batch')
    print("JAX evaluation step function defined and pmapped.")
else:
    print("Train state not created, cannot pmap evaluation step.")


# --- Training and Evaluation Loops ---
# Function to replicate and shard data for pmap
def shard_batch(batch):
    # Reshape batch for pmap: [batch_size] -> [num_devices, batch_size_per_device]
    # Assumes batch_size is divisible by num_devices
    num_devices = jax.local_device_count()
    # Ensure batch size is divisible by number of devices
    if batch['input_ids'].shape[0] % num_devices != 0:
        raise ValueError(f"Batch size ({batch['input_ids'].shape[0]}) must be divisible by the number of devices ({num_devices}) for pmap.")

    return jax.tree_util.tree_map(
        lambda x: x.reshape(num_devices, x.shape[0] // num_devices, *x.shape[1:]),
        batch
    )

# Check if 'state' and pmapped functions are defined before starting loops
if 'state' in locals() and 'pmapped_train_step' in locals() and 'pmapped_eval_step' in locals():
    print("Starting training...")
    train_metrics = []
    eval_metrics = []

    # Convert datasets to lists of NumPy arrays upfront to avoid as_numpy_iterator() RuntimeError
    print("Converting datasets to NumPy arrays...")
    try:
        train_data_np = list(train_dataset.as_numpy_iterator())
        eval_data_np = list(eval_dataset.as_numpy_iterator())
        print("Datasets converted to NumPy arrays.")
    except Exception as e:
         print(f"Error converting datasets to NumPy arrays: {e}")
         train_data_np = None
         eval_data_np = None


    if train_data_np is not None and eval_data_np is not None:
        # Replication of dropout key for pmap
        key = jax.random.PRNGKey(0) # Re-initialize key for training loop
        dropout_key_replicated = jax.device_put_replicated(key, jax.local_devices())

        # Determine total number of steps to iterate based on num_train_steps
        # The training loop should run for num_train_steps iterations
        # Check if num_train_steps is defined
        if 'num_train_steps' in locals():
            # Use a simple index for iterating over the NumPy data
            train_data_index = 0

            for step in range(num_train_steps):
                # Get a batch and shard it for pmap
                # Handle wrapping around the dataset for multiple epochs if needed
                if train_data_index >= len(train_data_np):
                    train_data_index = 0 # Reset index for next epoch
                    print(f"End of training dataset at step {step}. Restarting from beginning.")

                train_batch = train_data_np[train_data_index]
                train_data_index += 1

                sharded_train_batch = shard_batch(train_batch)


                # Perform a training step
                # Check if pmapped_train_step is defined
                if 'pmapped_train_step' in locals():
                    state, loss, accuracy, dropout_key_replicated = pmapped_train_step(
                        state,
                        sharded_train_batch,
                        dropout_key_replicated # Pass the replicated dropout key
                    )

                    # Log metrics periodically
                    if (step + 1) % SAVE_SUMMARY_STEPS == 0 or step == num_train_steps - 1: # Also log at the last step
                        # Gather metrics from all devices and take the mean
                        mean_loss = jnp.mean(loss)
                        mean_accuracy = jnp.mean(accuracy)
                        print(f"Step {step+1}/{num_train_steps}, Train Loss: {mean_loss:.4f}, Train Accuracy: {mean_accuracy:.4f}")
                        train_metrics.append({'step': step + 1, 'loss': float(mean_loss), 'accuracy': float(mean_accuracy)}) # Convert to float for list


                        # Run evaluation periodically
                        print("Running evaluation...")
                        eval_losses = []
                        eval_accuracies = []
                        # Iterate over the evaluation dataset (NumPy list)
                        # Check if eval_data_np is defined
                        if eval_data_np is not None:
                            # Replicate key for evaluation - not needed with deterministic=True passed directly
                            # eval_key_replicated = jax.device_put_replicated(jax.random.PRNGKey(step), jax.local_devices()) # Use step for unique key per eval

                            for eval_batch in eval_data_np:
                                # Check if pmapped_eval_step is defined
                                if 'pmapped_eval_step' in locals():
                                    sharded_eval_batch = shard_batch(eval_batch)
                                    # Pass a dummy key or handle rngs={'dropout': None} if deterministic=True
                                    loss, accuracy, _ = pmapped_eval_step(state, sharded_eval_batch) # Removed key argument here
                                    eval_losses.append(jnp.mean(loss))
                                    eval_accuracies.append(jnp.mean(accuracy))
                                else:
                                    print("Skipping evaluation step: pmapped_eval_step not defined.")
                                    break # Exit inner evaluation loop


                            # Calculate mean evaluation metrics
                            mean_eval_loss = jnp.mean(jnp.stack(eval_losses)) if eval_losses else float('nan')
                            mean_eval_accuracy = jnp.mean(jnp.stack(eval_accuracies)) if eval_accuracies else float('nan')

                            print(f"Evaluation Results at step {step+1}: Eval Loss: {mean_eval_loss:.4f}, Eval Accuracy: {mean_eval_accuracy:.4f}")
                            eval_metrics.append({'step': step + 1, 'loss': float(mean_eval_loss), 'accuracy': float(mean_eval_accuracy)}) # Convert to float for list
                        else:
                            print("Skipping evaluation: eval_data_np is None.")

                else:
                    print("Skipping training step: pmapped_train_step not defined.")
                    break # Exit training loop

            print("Training finished.")
        else:
            print("Skipping training loop: num_train_steps not defined.")
    else:
        print("Skipping training and evaluation loops: train_data_np or eval_data_np is None.")
else:
    print("Skipping training and evaluation loops because state or pmapped functions were not created.")


# --- Prediction ---
# Define the prediction step function (similar to eval_step but without loss/accuracy)
@partial(jax.jit) # Removed static_argnums
def predict_step(state, batch): # Removed dropout_key parameter here
     # Model forward pass in prediction mode
    # Check if state is defined before applying
    if 'state' in locals():
        logits = state.apply_fn(
            batch['input_ids'],
            batch['attention_mask'],
            batch['token_type_ids'],
            params=state.params, # Use trained parameters
            deterministic=True, # Explicitly pass deterministic as keyword argument
        )
        predicted_labels = jnp.argmax(logits, axis=-1)
        return predicted_labels, logits # Return logits for probabilities if needed
    else:
        print("Skipping prediction step: state not defined.")
        return None, None


# pmap the prediction step
# Check if 'state' is defined before pmapping
if 'state' in locals():
    pmapped_predict_step = jax.pmap(predict_step, axis_name='batch')
    print("\nJAX prediction step function defined and pmapped.")
else:
     print("\nTrain state not created, cannot pmap prediction step.")


# Run predictions
# Check if pmapped_predict_step is defined before running prediction loop
if 'pmapped_predict_step' in locals():
    print("Running predictions...")
    # Use the 'test' split of the tokenized dataset
    # Check if tokenized_dataset is defined and has a 'test' split
    if 'tokenized_dataset' in locals() and "test" in tokenized_dataset:
        test_dataset_for_prediction = tokenized_dataset["test"].to_tf_dataset(
            columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'], # Include labels if available (useful for verification)
            shuffle=False, # No shuffling for prediction
            batch_size=PREDICT_BATCH_SIZE,
            collate_fn=None,
            drop_remainder=True # Drop remainder for fixed shape in pmap
        )

        # Convert test dataset to list of NumPy arrays upfront
        print("Converting test dataset to NumPy arrays...")
        try:
            test_data_np = list(test_dataset_for_prediction.as_numpy_iterator())
            print("Test dataset converted to NumPy arrays.")
        except Exception as e:
            print(f"Error converting test dataset to NumPy arrays: {e}")
            test_data_np = None


        all_predictions = []
        all_logits = []

        # Use the list of NumPy arrays for prediction
        if test_data_np is not None:
            # Determine the number of prediction batches
            num_predict_batches = len(test_data_np)

            # Replicate key for prediction - not needed with deterministic=True passed directly
            # predict_key = jax.random.PRNGKey(0) # Use a fixed key for prediction
            # predict_key_replicated = jax.device_put_replicated(predict_key, jax.local_devices())


            for predict_step_idx in range(num_predict_batches):
                predict_batch = test_data_np[predict_step_idx]
                sharded_predict_batch = shard_batch(predict_batch)

                # Perform prediction step
                # Check if pmapped_predict_step is defined
                if 'pmapped_predict_step' in locals():
                    # Pass a dummy key or handle rngs={'dropout': None} if deterministic=True
                    sharded_predicted_labels, sharded_logits = pmapped_predict_step(state, sharded_predict_batch) # Removed key argument here

                    # Gather predictions from all devices and flatten
                    # Ensure correct reshaping and slicing based on the original batch size
                    # Check if sharded_predicted_labels and sharded_logits is not None
                    if sharded_predicted_labels is not None and sharded_logits is not None:
                        predicted_labels = sharded_predicted_labels.reshape(-1)[:sharded_predict_batch['input_ids'].shape[0] * sharded_predict_batch['input_ids'].shape[1] // jax.local_device_count()]
                        logits = sharded_logits.reshape(-1, sharded_logits.shape[-1])[:sharded_predict_batch['input_ids'].shape[0] * sharded_predict_batch['input_ids'].shape[1] // jax.local_device_count(), :]

                        all_predictions.extend(predicted_labels.tolist())
                        all_logits.extend(logits.tolist())
                    else:
                         print("Skipping prediction batch: sharded_predicted_labels or sharded_logits is None.")

                else:
                    print("Skipping prediction step: pmapped_predict_step not defined.")
                    break # Exit prediction loop

        else:
            print("Skipping prediction loop: test_data_np is None.")

        print("Predictions finished.")
        # Display sample predictions and their corresponding logits
        print("\nSample Predictions (first 10):")
        print(all_predictions[:10])
        print("\nSample Logits (first 10 predictions):")
        print(all_logits[:10])
    else:
        print("Skipping prediction: tokenized_dataset or test split not defined.")
else:
    print("\nSkipping prediction loop because state or pmapped prediction function was not created.")